# 📘 Galileo + OpenTelemetry Tracing Guide (Python)

This notebook demonstrates how to send OpenTelemetry traces to Galileo using OpenInference.

🔗 See full documentation [here](https://v2galileo.mintlify.app/guides/logs/otel)

In [ ]:
!pip install -U \
    groq \
    openinference-instrumentation-groq \
    opentelemetry-sdk \
    opentelemetry-exporter-otlp \
    opentelemetry-api \
    "opentelemetry-proto>=1.12.0" -q

In [ ]:
GALILEO_API_KEY = "YOUR_API_KEY"
PROJECT_NAME = "YOUR-PROJECT-NAME"
LOG_STREAM_NAME = "my_log_stream"



## 🔐 Authentication and Header Setup

In [ ]:
import os

headers = {
    "Galileo-API-Key": GALILEO_API_KEY,
    "project": PROJECT_NAME,
    "logstream": LOG_STREAM_NAME,
}
# Format headers as a comma-separated key=value string
# Example: Galileo-API-Key=abc123,project_name=my_project,log_stream_name=my_run
os.environ['OTEL_EXPORTER_OTLP_TRACES_HEADERS'] = ",".join([f"{k}={v}" for k, v in headers.items()])

## ⚙️ OpenTelemetry Setup

In [ ]:
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

from openinference.instrumentation.groq import GroqInstrumentor
endpoint = "https://app.galileo.ai/api/galileo/otel/traces"
tracer_provider = trace_sdk.TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

GroqInstrumentor().instrument(tracer_provider=tracer_provider)

OTEL tracing initialized.


In [ ]:
from groq import AsyncGroq, Groq
from groq.types.chat import ChatCompletionToolMessageParam

client = Groq(
        api_key=os.environ.get("GROQ_API_KEY"),
    )

    weather_function = {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "finds the weather for a given city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city to find the weather for, e.g. 'London'",
                    }
                },
                "required": ["city"],
            },
        },
    }

    sys_prompt = "Respond to the user's query using the correct tool."
    user_msg = "What's the weather like in San Francisco?"

    messages = [{"role": "system", "content": sys_prompt}, {"role": "user", "content": user_msg}]
    response = client.chat.completions.create(
        model="mixtral-8x7b-32768",
        messages=messages,
        temperature=0.0,
        tools=[weather_function],
        tool_choice="required",
    )

    message = response.choices[0].message
    assert (tool_calls := message.tool_calls)
    tool_call_id = tool_calls[0].id
    messages.append(message)
    messages.append(
        ChatCompletionToolMessageParam(content="sunny", role="tool", tool_call_id=tool_call_id),
    )
    final_response = client.chat.completions.create(
        model="mixtral-8x7b-32768",
        messages=messages,
    )